This code creates a Torch Dataset of predictors and targets for HAL_GRU to run

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import os.path



In [12]:
def create_time_delayed_dataset(predictors, targets, lookback = 300):
    dataX, dataY = [], []
    for i in range(len(targets)-lookback-1):
        predictor = predictors[i:(i+lookback), :]
        target = targets[i + lookback]
        dataX.append(predictor)
        dataY.append(target)

    dataX = torch.tensor(dataX)
    dataY = torch.tensor(dataY)
    #Convert to float tensors
    return dataX, dataY

In [28]:
participant_id_range = range(1,16)
tool_id_range = range(1,3)
trial_id_range = range(1,4)
output_directory = r'C:\Users\anand\Desktop\HAL Pytorch Datasets'
# Create a dataset for time-series modeling with a Gated Recurrent Unit (GRU)
# This dataset reads data from the HAL Labelled Data directory
# Creating separate datasets for each participant, tool, and hand
# The columns of the csv files are
#Time (s), Left Palm Force (N), Left Thumb Force (N), Left Index Force (N), Left Middle Force (N),Left Ring Force (N),
#Left Little Force (N),	Right Palm Force (N), Right Thumb Force (N), Right Index Force (N), Right Middle Force (N), 
#Right Ring Force (N), Right Little Force (N), Left HAL	Right HAL

# The dataset consists of 6 predictors, 1 total hand force = sum of all forces and 5 finger forces
# The dataset is initially downsampled by deleting half of the rows
# Then, 6 predictors with 300 samples are used to predict the 301th HAL score.
# Then, these predictors are moved down by one row and rows 2-301 are used to predict 302th HAL score
# Save a separate dataset for each participant and tool combo. Eg: p1_tool1.pt
for participant_id in participant_id_range:
    for tool_id in tool_id_range:
        left_dataset = {}
        right_dataset = {}  
        for trial_id in trial_id_range:            
            file_name = r'C:\Users\anand\Desktop\HAL Labelled Data\p'+str(participant_id)+" tool"+str(tool_id)+" trial"+str(trial_id)+' w lr HAL.csv'
            # Create an empty tensor dataset

            if os.path.isfile(file_name):
                print('Participant: ',participant_id,' Tool: ',tool_id,' Trial: ',trial_id)
                df_temp = pd.read_csv(file_name)
                # Delete every alternate row
                df_temp = df_temp.iloc[::2]
                left_predictor_names = ['Left Palm Force (N)','Left Thumb Force (N)','Left Index Force (N)','Left Middle Force (N)','Left Ring Force (N)','Left Little Force (N)']
                left_target_name = 'Left HAL'
                left_force_sum = df_temp[left_predictor_names].sum(axis=1)
                left_finger_forces = df_temp[left_predictor_names[1:]]
                left_predictors = np.array(pd.concat([left_force_sum,left_finger_forces],axis=1))
                left_target = np.array(df_temp[left_target_name])
                left_dataX, left_dataY = create_time_delayed_dataset(left_predictors, left_target)
                left_dataset[str(participant_id)+'_'+str(tool_id)+'_'+str(trial_id)] = (left_dataX, left_dataY)

                right_predictor_names = ['Right Palm Force (N)','Right Thumb Force (N)','Right Index Force (N)','Right Middle Force (N)','Right Ring Force (N)','Right Little Force (N)']
                right_target_name = 'Right HAL'
                right_force_sum = df_temp[right_predictor_names].sum(axis=1)
                right_finger_forces = df_temp[right_predictor_names[1:]]
                right_predictors = np.array(pd.concat([right_force_sum,right_finger_forces],axis=1))
                right_target = np.array(df_temp[right_target_name])
                right_dataX, right_dataY = create_time_delayed_dataset(right_predictors, right_target)
                right_dataset[str(participant_id)+'_'+str(tool_id)+'_'+str(trial_id)] = (right_dataX, right_dataY)
                # Concatenate datasets with the same tool id and hand
        # Save the dataset
        left_file_name = 'p'+str(participant_id)+'_tool'+str(tool_id)+'_left.pt'
        left_predictors_combined = torch.cat([left_dataset[key][0] for key in left_dataset.keys()])
        left_target_combined = torch.cat([left_dataset[key][1] for key in left_dataset.keys()])
        left_XY = torch.utils.data.TensorDataset(left_predictors_combined, left_target_combined)
        torch.save(left_XY,os.path.join(output_directory,left_file_name))

        right_file_name = 'p'+str(participant_id)+'_tool'+str(tool_id)+'_right.pt'
        right_predictors_combined = torch.cat([right_dataset[key][0] for key in right_dataset.keys()])
        right_target_combined = torch.cat([right_dataset[key][1] for key in right_dataset.keys()])
        right_XY = torch.utils.data.TensorDataset(right_predictors_combined, right_target_combined)
        torch.save(right_XY,os.path.join(output_directory,right_file_name))



                
            
    

Participant:  1  Tool:  1  Trial:  1
Participant:  1  Tool:  1  Trial:  2
Participant:  1  Tool:  1  Trial:  3
Participant:  1  Tool:  2  Trial:  1
Participant:  1  Tool:  2  Trial:  2
Participant:  1  Tool:  2  Trial:  3
Participant:  2  Tool:  1  Trial:  1
Participant:  2  Tool:  1  Trial:  2
Participant:  2  Tool:  1  Trial:  3
Participant:  2  Tool:  2  Trial:  1
Participant:  2  Tool:  2  Trial:  2
Participant:  2  Tool:  2  Trial:  3
Participant:  3  Tool:  1  Trial:  1
Participant:  3  Tool:  1  Trial:  2
Participant:  3  Tool:  1  Trial:  3
Participant:  3  Tool:  2  Trial:  1
Participant:  3  Tool:  2  Trial:  2
Participant:  4  Tool:  1  Trial:  1
Participant:  4  Tool:  1  Trial:  2
Participant:  4  Tool:  2  Trial:  1
Participant:  4  Tool:  2  Trial:  2
Participant:  5  Tool:  1  Trial:  1
Participant:  5  Tool:  1  Trial:  2
Participant:  5  Tool:  2  Trial:  1
Participant:  5  Tool:  2  Trial:  2
Participant:  6  Tool:  1  Trial:  1
Participant:  6  Tool:  1  Trial:  2
P

In [27]:
left_predictors_combined.shape

torch.Size([23887, 300, 6])